# Data Exploration

In [1]:
# import needed packages
import pandas as pd
import sqlite3
from pandasql import sqldf

In [2]:
# connect to database
conn = sqlite3.connect('challenge.db')

# read in tables
accessTable = "access"
access = pd.read_sql_query(f"SELECT * FROM {accessTable}", conn)
citiesTable = "five_hundred_cities"
cities = pd.read_sql_query(f"SELECT * FROM {citiesTable}", conn)
varsTable = "variable_list"
vars = pd.read_sql_query(f"SELECT * FROM {varsTable}", conn)

# close connection
conn.close()

In [3]:
# read in extra data
socioeconomic = pd.read_excel('data\\FoodEnvironmentAtlas.xls', sheet_name='SOCIOECONOMIC') # for filtering to high elder populations
countyData = pd.read_excel('data\\FoodEnvironmentAtlas.xls', sheet_name='Supplemental Data - County') # for impact analysis

# Initial Filtering
Members of a medicare advantage plan will be 65 years or older, so at least for initial analysis it is best to look at the locations with the highest elderly populations.

In [5]:
# find states with a higher average % of 65+ than the overall average
query = """
SELECT State, AVG(PCT_65OLDER10) as AVG_PCT_65OLDER10
FROM socioeconomic
GROUP BY State
HAVING AVG_PCT_65OLDER10 >= (SELECT AVG(PCT_65OLDER10) as AVG_PCT_65OLDER10
                                FROM socioeconomic)
ORDER BY AVG_PCT_65OLDER10 DESC
"""
topStates = sqldf(query, globals())
topStates.head()

,State,AVG_PCT_65OLDER10
0,ND,20.191338
1,NE,19.327734
2,MT,18.258659
3,KS,18.090187
4,FL,18.073794


# Group Analysis
Possible Indicators of Food Insecurity:
1. Access issues (list vars of interest):
    - members without personal transportation: PCT_LACCESS_HHNV10
    - low income members: PCT_LACCESS_LOWI10
    - seniors: PCT_LACCESS_SENIORS10
2. Health conditions (list vars linked to food insecurity):
    - high blood pressure: BPHIGH_CrudePrev
    - heart disease: CHD_CrudePrev
    - diabetes: DIABETES_CrudePrev
    - high cholesterol: HIGHCHOL_CrudePrev
    - obesity: OBESITY_CrudePrev

note: comparing % in 2010

In [6]:
# filter the access data to only top states
# include location information and the % groups of interest
query = """
SELECT FIPS, State, County, PCT_LACCESS_HHNV10, PCT_LACCESS_LOWI10, PCT_LACCESS_SENIORS10
FROM access
WHERE State IN (SELECT State FROM topStates);
"""
accessFil = sqldf(query, globals())
accessFil.head()

,FIPS,State,County,PCT_LACCESS_HHNV10,PCT_LACCESS_LOWI10,PCT_LACCESS_SENIORS10
0,4001,AZ,Apache,11.935694,43.828039,7.848549
1,4003,AZ,Cochise,2.959606,12.335635,7.686568
2,4005,AZ,Coconino,2.833944,13.095848,3.356048
3,4007,AZ,Gila,3.529097,11.545643,5.654099
4,4009,AZ,Graham,4.756930,13.702319,3.368028


In [7]:
# filter the cities data to only top states
# include location information and the groups of interest
query = """
SELECT StateAbbr, PlaceName, PlaceFIPS, TractFIPS, Population2010, Geolocation,
        BPHIGH_CrudePrev, CHD_CrudePrev, DIABETES_CrudePrev, HIGHCHOL_CrudePrev,
        MHLTH_CrudePrev, OBESITY_CrudePrev, PHLTH_CrudePrev
FROM cities
WHERE StateAbbr IN (SELECT State FROM topStates);
"""
citiesFil = sqldf(query, globals())
citiesFil.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Population2010,Geolocation,BPHIGH_CrudePrev,CHD_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PHLTH_CrudePrev
0,AZ,Avondale,404720,4013061013,1215,"(33.49776276980, -112.324917007)",18.1,2.0,5.6,21.9,11.4,24.6,7.9
1,AZ,Avondale,404720,4013061014,4905,"(33.48595366650, -112.328154888)",22.4,3.3,7.3,26.1,13.4,29.8,10.7
2,AZ,Avondale,404720,4013061015,5757,"(33.47210439560, -112.331415748)",24.5,3.8,7.8,29.1,12.1,28.8,10.6
3,AZ,Avondale,404720,4013061021,1753,"(33.48478404890, -112.348211384)",20.6,2.8,6.3,25.6,10.9,25.8,8.6
4,AZ,Avondale,404720,4013061200,6114,"(33.44623630350, -112.341766281)",28.6,6.0,12.4,31.0,18.0,36.0,16.9


In [8]:
# adding leading zeros to county FIPS codes
accessFil['FIPS'] = accessFil['FIPS'].astype(str)
accessFil['FIPS'] = accessFil['FIPS'].apply(lambda x: '0' + x if len(x) == 4 else x)

# adding leading zeros to county FIPS codes
citiesFil['TractFIPS'] = citiesFil['TractFIPS'].astype(str)
citiesFil['TractFIPS'] = citiesFil['TractFIPS'].apply(lambda x: '0' + x if len(x) == 10 else x)

# create a countyFIPS column in the cities data
citiesFil['CountyFIPS'] = citiesFil['TractFIPS'].str[:5]

In [9]:
# join the access and cities data on the county FIPS code
query = """SELECT *
FROM citiesFil INNER JOIN accessFil
ON citiesFil.CountyFIPS = accessFil.FIPS;
"""
combinedTop = sqldf(query, globals())
combinedTop.head()

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Population2010,Geolocation,BPHIGH_CrudePrev,CHD_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PHLTH_CrudePrev,CountyFIPS,FIPS,State,County,PCT_LACCESS_HHNV10,PCT_LACCESS_LOWI10,PCT_LACCESS_SENIORS10
0,AZ,Avondale,404720,04013061013,1215,"(33.49776276980, -112.324917007)",18.1,2.0,5.6,21.9,11.4,24.6,7.9,04013,04013,AZ,Maricopa,0.78454,3.667382,2.029047
1,AZ,Avondale,404720,04013061014,4905,"(33.48595366650, -112.328154888)",22.4,3.3,7.3,26.1,13.4,29.8,10.7,04013,04013,AZ,Maricopa,0.78454,3.667382,2.029047
2,AZ,Avondale,404720,04013061015,5757,"(33.47210439560, -112.331415748)",24.5,3.8,7.8,29.1,12.1,28.8,10.6,04013,04013,AZ,Maricopa,0.78454,3.667382,2.029047
3,AZ,Avondale,404720,04013061021,1753,"(33.48478404890, -112.348211384)",20.6,2.8,6.3,25.6,10.9,25.8,8.6,04013,04013,AZ,Maricopa,0.78454,3.667382,2.029047
4,AZ,Avondale,404720,04013061200,6114,"(33.44623630350, -112.341766281)",28.6,6.0,12.4,31.0,18.0,36.0,16.9,04013,04013,AZ,Maricopa,0.78454,3.667382,2.029047


In [10]:
# create function to find top 10 counties in food access challenge metrics
def accessTop10(data, metric):
    groupedData = data.groupby(['State', 'County'])[metric].mean().reset_index()
    sortedData = groupedData.sort_values(by=metric, ascending=False).head(n=10)
    sortedData['Population Group'] = metric
    sortedData.rename(columns={metric:'Avg %'}, inplace=True)
    return(sortedData)

# get top 10 counties with high food access challenges
noCar = accessTop10(combinedTop, 'PCT_LACCESS_HHNV10')
lowIncome = accessTop10(combinedTop, 'PCT_LACCESS_LOWI10')
lowSeniors = accessTop10(combinedTop, 'PCT_LACCESS_SENIORS10')

# combine challenges into one data frame
foodAccessTop = pd.concat([noCar,lowIncome,lowSeniors])
foodAccessTop.head()

,State,County,Avg %,Population Group
124,WV,Kanawha,5.032939,PCT_LACCESS_HHNV10
60,MN,St. Louis,3.364051,PCT_LACCESS_HHNV10
85,OK,Osage,3.214825,PCT_LACCESS_HHNV10
86,OK,Pottawatomie,3.184618,PCT_LACCESS_HHNV10
50,MI,Ingham,3.097150,PCT_LACCESS_HHNV10


In [11]:
# create function to find top 10 counties in health conditions
def healthTop10(data, metric):
    groupedData = data.groupby(['State', 'County'])[metric].mean().reset_index()
    sortedData = groupedData.sort_values(by=metric, ascending=False).head(n=10)
    sortedData['Health Condition'] = metric
    sortedData.rename(columns={metric:'Avg Prevalance'}, inplace=True)
    return(sortedData)

# health conditions
bpHigh = healthTop10(combinedTop, 'BPHIGH_CrudePrev')
heartDisease = healthTop10(combinedTop, 'CHD_CrudePrev')
diabetes = healthTop10(combinedTop, 'DIABETES_CrudePrev')
highChol = healthTop10(combinedTop, 'HIGHCHOL_CrudePrev')
obesity = healthTop10(combinedTop, 'OBESITY_CrudePrev')

# combine health conditions into one data frame
lowHealthTop = pd.concat([bpHigh,heartDisease,diabetes,highChol,obesity])
lowHealthTop.head()

,State,County,Avg Prevalance,Health Condition
85,OK,Osage,48.600000,BPHIGH_CrudePrev
49,MI,Genesee,45.907692,BPHIGH_CrudePrev
56,MI,Wayne,44.233514,BPHIGH_CrudePrev
124,WV,Kanawha,44.022727,BPHIGH_CrudePrev
86,OK,Pottawatomie,43.200000,BPHIGH_CrudePrev


In [12]:
# find repeat counties within both data sets
accessCounts = foodAccessTop[['State', 'County']].value_counts()
healthCounts = lowHealthTop[['State', 'County']].value_counts()

# find repeat counties between both data sets
repeatCounties = pd.merge(foodAccessTop, lowHealthTop, on=['State','County'], how='inner')
repeatCounties = repeatCounties[['State','County']].drop_duplicates()
repeatCounties

,State,County
0,WV,Kanawha
5,OK,Osage
10,OK,Pottawatomie
14,KS,Wyandotte
17,PA,Erie
26,FL,St. Lucie
27,FL,Flagler
31,FL,Lee
33,FL,Polk
35,IL,Macon


# Next Steps: Impact Assessment
- using only the repeat counties, project the population changes 2010 - 2018
- for health conditions get the prevalence based on the 2010 population and calculate the population counts for the following years (2011 - 2018)
- for low access groups look at the percentage change from 2010 - 2015 to assess which group will be most affected


In [13]:
# remove the word county from the county column
countyData['County'] = countyData['County'].str.replace(' County$', '', regex=True)

# replace abbrevations with full names
stateFull = {
    'FL': 'Florida',
    'IL': 'Illinois',
    'KS': 'Kansas', 
    'OK': 'Oklahoma',
    'PA': 'Pennsylvania',
    'WV': 'West Virginia'
}
repeatCounties['State'] = repeatCounties['State'].replace(stateFull)

# remove any extra spaces
countyData['County'] = countyData['County'].str.strip()
repeatCounties['County'] = repeatCounties['County'].str.strip()
countyData['State'] = countyData['State'].str.strip()
repeatCounties['State'] = repeatCounties['State'].str.strip()

# filter to only repeated counties
countiesFil = pd.merge(repeatCounties, countyData, on=['State','County'], how='inner')
countiesFil.head()

,State,County,FIPS,2010_Census_Population,Population_Estimate_2011,Population_Estimate_2012,Population_Estimate_2013,Population_Estimate_2014,Population_Estimate_2015,Population_Estimate_2016,Population_Estimate_2017,Population_Estimate_2018
0,West Virginia,Kanawha,54039,193063,192157,192234,191530,190353,188270,186162,183310,180454
1,Oklahoma,Osage,40113,47472,47783,47463,47394,47494,47342,47378,47328,47014
2,Oklahoma,Pottawatomie,40125,69442,70045,70514,70907,71539,71495,72041,72248,72679
3,Kansas,Wyandotte,20209,157505,158020,159411,160984,162320,163832,164934,165313,165324
4,Pennsylvania,Erie,42049,280566,281296,281523,280724,279437,278182,276289,273892,272061


# Future Idea: Association Analysis
**Among the repeated counties, determine what instances (ex: low income, high blood pressure) are likely to occur together.